This notebook explains the code in `create_drug_combo_person_time.py`, including an example of one drug, to show how the feature matrix is created and how the weights get added. We have divided up the code from the main function `one_basedrug` to show the purpose of each chunk (denoted "block").

All patient data shown here are not real patient data, but they represent scrambled versions of the data, for the purpose of an example.

In [1]:
import regression_util as rs
import load_subject_info as his2ft
import util
import create_weights as wt

drugid = 4565
hisdir = 'hisagain-12/'
resdir = 'dcomb/'

outcomes = "outcomes_no_nonmel.pkl"
ctl_outcomes_name = 'ctloutcomes_match.json'

foutcomes = "outcomes_no_nonmel.pkl"
TIME_CHUNK = 12
fut_den_feat=False
drug1_washout=20
drug_any_washout=np.inf
FILT_ERA = 0
combos_do = "pairs_todo.json"
agg = 2
import json

## 1. Setting up the  cohort of patients taking drug A

#### Block 1: get all patients taking drug A
First, make list of all patients who are taking drug A, and save this into a directory 

In [2]:
### Block 1
comboname = util.combo_names2(resdir, drugid)

## get patients who took drug A
trtid_file = comboname + "trtid"    

if not os.path.exists(trtid_file):
    his2ft.write_ids(hisdir, drugid, trtid_file)

trtid = np.loadtxt(trtid_file,dtype=int)
### end Block 1
trtid[:5]

array([ 6563, 14446, 18588, 18648, 49817])

#### Block 2: Split up patients into subcohorts
Split up patients who took drug A into cohorts based on a heuristic so that the person-time matrix is not giant.

The matrix gets bigger with:
- smaller windows of time (the smallest in our implementation is 3 months, but we aggregate this to 6 month windows)
- no censoring for discontinuation of drug 1 (defined in "drug1_washout")

Once we split the full list into smaller cohorts, this code gives us a list of files containing the IDs of patients in each cohort

In [3]:
### Block 2
BIGNESS = 300000/max(1,3-agg, (drug1_washout - 20) % (agg*12))
if drug1_washout == np.inf:
    BIGNESS = 100000

NSAMP = 5
BFRAC = 1.2
idfiles =  []
#pdb.set_trace()
suff2 = wt.suff(agg, FILT_ERA, drug1_washout,drug_any_washout).strip(".")

### split up patients who took drug A into cohorts based on a heuristic so that the person-time matrix is not giant
if len(trtid) > BFRAC*BIGNESS:
    truesplits = int(np.ceil(len(trtid)/BIGNESS))
    trt_splits = min(truesplits, NSAMP)
    trt_split_index = rs.get_splitvec(len(trtid), truesplits)
    for i in range(trt_splits):
        #fname = comboname + "spl" + str(i)
        fname = comboname + "spl" + str(i) + suff2 
        if not os.path.exists(fname):
            tid = trtid[trt_split_index==i]
            if tid.shape[0] > 800000:
                tid = np.random.choice(tid, size=800000, replace=False)
            np.savetxt(fname,tid)
        else:
            idfiles = sorted(glob.glob(comboname + "spl*" +suff2))
            break
        idfiles.append(fname)
else:
    idfiles = [comboname + 'trtid']
print("splits ",",".join(idfiles))        
del trtid
### end Block 2


splits  dcomb/Combo2.4565/trtid


Here is the result for this drug, which is not so common so only one cohort:

In [17]:
idfiles

['dcomb/Combo2.4565/trtid']

#### Block 3: Set up the outcomes and the features (confounders)

Here we obtain the different outcomes (cancers or negative control outcomes), as well as the codes that we use to identify these phenotypes

In [4]:
### Block 3

## matching codes to their meanings
voc = pd.read_pickle("../../data/clid.vi.allvocab.pkl")

## outcomes = dictionary of {disease name:[medical code list] }
outcomes = sorted(pickle.load(open(outcomes,'rb')).keys())


print(outcomes)

['Bladder_Cancer', 'Bone_Cancer', 'Brain_Cancer', 'Breast_Cancer', 'CNS_Cancer', 'Cancer_history', 'Carinoid_Neoplasm', 'Cervical_Cancer', 'Colorectal_Cancer', 'Connective_Tissue_Cancer', 'Endocrine_Cancer', 'Esophageal_Cancer', 'Eye_Cancer', 'Female_Genital_Cancer', 'Generalized_Digestive_Cancer', 'Hepatobiliary_Cancer', 'Hereditary_Breast_Cancer', 'Intestinal_Cancer', 'Intrathoracic_Organ_Cancer', 'Kidney_Cancer', 'Lung_Cancer', 'Male_Breast_Cancer', 'Male_Genital_Cancer', 'Obs_uspected_neoplasm', 'Oro_Naso_Pharyngeal_Cancer', 'Other_Lymphoid_Histiocytic_Cancers', 'Ovarian_Cancer', 'Pancreatic_Cancer', 'Prostate_Cancer', 'Respiratory_System_Cancer', 'Secondary_Malignant_Neoplasm', 'Stomach_Cancer', 'Thyroid_Cancer', 'Unspecified_Cancer', 'Uterine_Cancer']


In [6]:
### negative control outcomes
### ctl_outcomes = dict of {'outcome name': [list of vocab ids]}
ctl_outcomes = json.loads(open(ctl_outcomes_name).read())
ctl_outcomes_name = ctl_outcomes_name.split(".")[0]
ctlo = sorted(ctl_outcomes.keys())

## this parts 
outcomes_list =  [ctl_outcomes[i] for i in ctlo]
print(ctlo)

print(outcomes_list)

### end Block 3

['Arterial_Embolism_Thrombosis', 'E Codes: Other specified; NEC', 'Emphysema_COPD', 'General_Dementia', 'General_Paralysis', 'HIV', 'Inflammatory_Toxic_Neuropathy', 'Myocardial_Infarction', 'Obese_bmi', 'Pancytopenia', 'Parkinsons_Disease', 'Personality_Disorder', 'Septic_Arthritis', 'Septicemia', 'Spleen_Disease', 'Ulcerative_Colitis', 'Unspecified_Autoimmune_Disease', 'Unspecified_Hepatitis', 'Unspecified_Immune_Deficiency']
[[5398, 8009, 13365, 13369, 13379, 13389, 13401], [3610, 3674, 9823, 9825, 10028, 10231, 10716, 11068], [4615, 4616, 4617, 6867, 6868, 7318, 7332, 11300, 11301, 11303, 11304, 11591, 12071, 12270], [5421, 5422, 8882, 8883, 9378, 10205, 10206, 10209, 10210, 10859, 11476, 11478, 15308, 15309], [3169, 3171, 3172, 3173, 3174, 3175, 3606, 3607, 3608, 3609, 4914, 4917, 9764, 9765, 9766, 11168, 15223, 15645, 15753], [7359, 7762], [11182, 11183, 11185, 11188, 11189, 11190, 13418, 13419, 13713, 13852], [4503, 6315, 6318, 6586, 6749, 6750, 6751, 7174, 7175, 7176, 7664, 7678

#### Block 4: set up features of obtain: all features + all outcomes
We encode the medical histories in a sparse format, meaning for each patient, in each time window, we just have the medical codes that happened in this time window. We want to convert this to a feature matrix.

The "sparse index" represents the columns of this feature matrix that we will use to train our models for weighting.

Normally we only keep features that appear frequently, but we also add in our outcomes to make sure that we can record when those happen or if they have happened in the past (an exclusion criterion for the cohort study).

In [7]:
### Block 4

### get all of the possible health codes as features
sparse_ix_name = util.sparse_index_name(comboname)
if not os.path.exists(sparse_ix_name):
    his2ft.create_sparse_index(hisdir, drugid, sparse_ix_name)    

past_sparse_index = util.get_sparseindex(comboname)

rx_vi = set(voc.loc[voc['type']=='rx','vi'].values)
outcome_vi_keep = sorted(chain.from_iterable(outcomes_list))

### all these people have no history of cancer but may have histoy of the CTL outcomes
### thus we need to make sure we are recording their past CTL outcomes
past_inp = np.array(sorted(set(outcome_vi_keep) | set(past_sparse_index)))

### end Block 4

print(past_inp)

get_sparseindex: deleting zero
[    2     4     6 ... 30273 30277 30282]


## 2. Analysis per small cohort
Now we go into a for loop which prepares the cohort study info for each of these smaller cohorts.

`for runi,fname in enumerate(idfiles):`

Briefly, for each of these cohorts, we:
- estimate censoring weights (probability of discontinuing the first drug A)
- find all second drugs that are taken after the first drug (with some cutoff of frequency, such as 3000 people taking drug B after drug A).
- Then for each of these drug pairs, we will do the drug combination analysis, next section

In [8]:
runi = 0
fname = idfiles[runi]

#### Block 5: load cohort history

Load the data corresponding to the set of patients in this smaller cohort.

This function takes as input what the incident drug (drug A) is, the cohort of patients, and the list of features (medical codes) we wish to use.

It creates a set of feature matrices describing each person's past history before drug A, and their follow-up time after starting drug A, which may include things like: discontinuing drug A, starting drug B, incidence of one of our outcome diseases.

In [9]:
### Block 5
runi  = str(runi) +  "."
run_save = comboname + runi  + suff2 + "."
nopastuct = pd.DataFrame()
d2counts = run_save + ".counts.txt"

ret = his2ft.censored_sparsemat(hisdir,
                    past_inp, np.loadtxt(fname), drugid,
                    0, TIME_CHUNK, agg=agg,
                                washout=drug1_washout, any_washout=drug_any_washout,
                                keep_vi = outcome_vi_keep)
[dense, past, fut, lab, fut_sparse_ix, cens_info, ch_len, fut_ix_fit] = ret[:8]

past.data = np.clip(past.data,0,1) #past = past > 0
todo = set(fut_sparse_ix) & rx_vi ## this is for obtaining the drug2's


t1: make future index 6.18
Censmat: 16643 past elt & 3822 fut elt
CHUNK  c0, nrow=110990 0.47
END nrow=110990 0.78, time = 0.71 min


Here, we go through the data matrices created by this function. We have one row per person-time (here, 110,990 rows, with 40680 users of drug A). Person time is expanded to follow the person from initiation of drug A to censoring/end of record.
- `dense` has info about their gender, age, number of drugs, the calendar time, etc for each follow-up interval
- `past` is derived from all of the drugs, conditions, procedures that appear in this cohort (16643 total) *before* initiaton of drug A. Note that past this matrix is expanded to match the number of entries follow up intervals.
- `cens_info` has info about the various intervals and if the person is censored at that interval (Discontinues drug 1)
- `fut` this is analogous to `past` but has all of the info for each particular follow up window (where the windows are denoted in the `cens_info` matrix).

In [42]:
len(set(dense[:,0]))

40680

In [35]:
dense.shape

(110990, 9)

In [36]:
past.shape

(110990, 16643)

In [9]:
past_inp.shape

(16643,)

In [37]:
fut.shape

(110990, 3822)

In [41]:
cens_info.shape

(110990, 4)

In [33]:
pd.DataFrame(dense[:10,:],
            columns = ['id','interval_length','week','age','gender','# new drug','# dx','# rx','# px'] )

,id,interval_length,week,age,gender,# new drug,# dx,# rx,# px
0,1973.0,0.0,197.0,63.000000,0.0,2.0,4.0,3.0,5.0
1,1973.0,0.0,221.0,63.461538,0.0,2.0,4.0,3.0,5.0
2,1973.0,0.0,245.0,63.923077,0.0,2.0,4.0,3.0,5.0
3,1973.0,0.0,269.0,64.384615,0.0,2.0,4.0,3.0,5.0
4,4368.0,0.0,150.0,53.000000,1.0,5.0,68.0,10.0,20.0
5,4368.0,0.0,174.0,53.461538,1.0,5.0,68.0,10.0,20.0
6,5627.0,0.0,62.0,24.000000,1.0,2.0,18.0,8.0,8.0
7,5627.0,18.0,86.0,24.461538,1.0,2.0,18.0,8.0,8.0
8,5642.0,0.0,394.0,60.000000,1.0,11.0,46.0,41.0,80.0
9,15380.0,0.0,64.0,63.000000,1.0,1.0,3.0,4.0,8.0


In [11]:
cens_info[:10]

,ids,censored,interval_start,interval_end
0,1973.0,0,0.0,24.0
1,1973.0,0,24.0,48.0
2,1973.0,0,48.0,72.0
3,1973.0,1,72.0,96.0
4,4368.0,0,0.0,24.0
5,4368.0,1,24.0,48.0
6,5627.0,0,0.0,24.0
7,5627.0,0,24.0,42.0
8,5642.0,1,0.0,24.0
9,15380.0,0,0.0,24.0


The `past` and `fut` matrices are large (but sparse), which allow them to capture possible confounders.
The `past_inp` contains the info about what the columns are. 

Below, we use this to view small bits of the data (mosly zeros because the first 10 rows do not have records of these codes), using the index to obtain what the names of these features are.

In [16]:
pd.DataFrame(past[:10,:10].todense(),
             columns = voc.set_index('vi').loc[past_inp[:10],'name'])

name,cpm_methscopolamine_nitrate_ppa_hcl,insulin_aspart_insulin_aspart_protamine,montelukast_sodium,enalapril_maleate_felodipine,betamethasone_sodium_phosphate,naratriptan_hydrochloride,antipyrine_benzocaine,oprelvekin,amoxicillin_clavulanate_potassium,estradiol_acetate
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pd.DataFrame(past[:10,7000:7010].todense(),
             columns = voc.set_index('vi').loc[past_inp[7000:7010],'name'])

name,E Codes: Natural/environment,Gestational_Pregnancy_Related_Disorder,Poisoning|Poisoning by other medications and drugs,Poisoning|Poisoning by other medications and drugs,Other injuries and conditions due to external causes,Non_Specific_Congenital_Eye_Anomaly,Non_Specific_Congenital_Eye_Anomaly,E Codes: Fall,E Codes: Fall,Alveolar_Disease
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pd.DataFrame(past[:10,-10:].todense(),
             columns = voc.set_index('vi').loc[past_inp[-10:],'name'])

name,pxK0043,pxK0040,pxK0045,pxK0048,px84466,px01210,px01215,px84460,pxE0250,px87380
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Similar for the `fut` matrix which has the occurrence of different medical codes in each time perioda after drug A. Unlike the `past` matrix which has the same data repeated for different time periods for each patient (for computational purposes), the rows in the `fut` matrix are not repeated but represent unique data for each person-time point.

In [20]:
fut_sparse_ix[:10]

array([19974, 27148, 15386, 26146, 22565, 21030,  1574, 29735, 23591,
       22056])

In [21]:
fut_sample = pd.DataFrame(fut[:10,:10].todense(),
             columns = voc.set_index('vi').loc[fut_sparse_ix[:10],'name'])
fut_sample

name,px84520,px83036,Immunizations and screening for infectious disease,px83550,px82565,px85025,moxifloxacin_hydrochloride,pxV5020,px00740,px84550
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


To make clearer the correspondence between the time-specific data in `fut` and the other info about each time period, they are concatenated horizontally below.

For example, person 1973 is followed for 4 time intervals; censored in the last one; and also has procedure px83036 in the last one only:

In [35]:
pd.concat((cens_info[:10], fut_sample),axis=1)

,ids,censored,interval_start,interval_end,px84520,px83036,Immunizations and screening for infectious disease,px83550,px82565,px85025,moxifloxacin_hydrochloride,pxV5020,px00740,px84550
0,1973.0,0,0.0,24.0,0,0,0,0,0,1,0,0,0,0
1,1973.0,0,24.0,48.0,0,0,0,0,0,1,0,0,0,0
2,1973.0,0,48.0,72.0,0,0,0,0,0,0,1,0,0,0
3,1973.0,1,72.0,96.0,0,1,0,0,0,1,0,1,0,0
4,4368.0,0,0.0,24.0,0,0,0,0,0,0,0,0,0,0
5,4368.0,1,24.0,48.0,0,0,0,0,0,0,0,0,0,0
6,5627.0,0,0.0,24.0,0,0,0,0,0,0,0,0,0,0
7,5627.0,0,24.0,42.0,0,0,0,0,0,0,0,0,0,0
8,5642.0,1,0.0,24.0,0,0,0,0,0,0,0,0,0,0
9,15380.0,0,0.0,24.0,0,0,0,0,0,0,0,0,0,0


#### Block 6: count users of second drug

Now look to see how many users there of each other drug subsequently. We only try to estimate the effect if there are drugs more frequent than `drug_freq_filter` such as 8000 users of the combination. For our example we just chose 3000 users, which only leaves us with 1 drug to test.

In [37]:
drug_freq_filter = 3000
### Block 6
if not os.path.exists(d2counts):
    sel = np.where(np.isin(fut_sparse_ix, list(rx_vi)))[0]
    rxvin = fut_sparse_ix[sel]

    res = {}
    for i in range(len(sel)):
        if not rxvin[i] in past_sparse_index:
            continue
        idd=  pd.DataFrame({'ids':cens_info['ids'],
                            'past':np.array(past[:,np.where(past_inp==rxvin[i])[0]].todense())[:,0],
                            'drug2':np.array(fut[:,sel[i]].todense())[:,0]}).groupby('ids').agg(max)


        res[rxvin[i]] = ((idd['past']==0) & (idd['drug2'] > 0)).sum() #idd.sum().values[0]


    nopastuct = pd.DataFrame(res,index=['ct']).transpose().sort_values('ct',ascending=False)
    #nopastuct.to_pickle(run_save + ".counts.pkl")
    nopastuct.to_csv(d2counts,sep="\t")
#pdb.set_trace()
todo = list(nopastuct.loc[nopastuct['ct'] > drug_freq_filter].index)


In [38]:
todo

[1190]

In [40]:
past_sparse_index.shape

(16605,)

In [41]:
past_inp.shape

(16643,)

In [42]:
past.shape

(110990, 16643)

#### Block 7: Set up outcome history and feature matrices for past medical history
Our next step will be modeling the confounding of censoring. But first, we need to set up the data correctly.

We are using these big feature matrices for two separate purposes:
- find history of the disease outcomes of interest, and find when/if in follow-up the disease outcomes happen
- to record confounding medical history.

Here, we separate out the data needed for these two purposes. 
First we get the data containing the "history of outcome"-- people with history before drug A are not included in the study.

Then we keep the feature columns that are not just there for keeping track of history of outcomes

In [12]:
### Block 7
### we included some elements in history just to get if person has history
### of disease... might include sparse not good things, remove for regression
### just keep if person (row) has history of outcome
history_of = np.array([np.array(past[:,np.where(np.isin(past_inp,ovi))[0]].sum(axis=1)>0)[:,0]
                for ovi in outcomes_list]).transpose()

past = past[:,np.where(np.isin(past_inp, past_sparse_index))[0]]



In [13]:
history_of.shape

(110990, 19)

In [14]:
pd.DataFrame(history_of[:10,:], columns = ctlo)

,Arterial_Embolism_Thrombosis,E Codes: Other specified; NEC,Emphysema_COPD,General_Dementia,General_Paralysis,HIV,Inflammatory_Toxic_Neuropathy,Myocardial_Infarction,Obese_bmi,Pancytopenia,Parkinsons_Disease,Personality_Disorder,Septic_Arthritis,Septicemia,Spleen_Disease,Ulcerative_Colitis,Unspecified_Autoimmune_Disease,Unspecified_Hepatitis,Unspecified_Immune_Deficiency
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


#### Block 8: set up outcome incidence and follow-up time features
In block 8 we do the same thing, but for follow up time. We make 2 matrices:
- `fut_inc`: incidence of the outcomes in the time periods (we make it so 0 indicates no outcome so far; 1 indicates incident outcome; -1 indicates outcome happened in the past)
- `fut`: the feature matrix for each time window

In [16]:
### Block 8
## we also included outcomes in the follow-up features, though they might not be common enough to use for
##   regression-- we want to keep the info about when the outcomes happen, but remove it from the features for regression
## we get the future-incidences which correspond to our particular outcomes (outcomes_list = list of the Vocab Ids VI)
x  = pd.DataFrame(np.transpose(np.array([np.array(fut[:,np.where(np.isin(fut_sparse_ix,ovi))[0]].sum(axis=1)>0)[:,0]
                    for ovi in outcomes_list])),
                  columns = ctlo,dtype=int)
x = pd.concat((cens_info['ids'],x),axis=1)
y = x.groupby('ids').agg(np.cumsum)
y = pd.concat((x['ids'],y),axis=1)
z = y.groupby('ids').agg(np.cumsum)
fut_inc = z.mask(z > 1, other = -1)
fut_inc = fut_inc.mask(history_of, other=-1)
del history_of, x, y, z 

fut = fut[:,np.isin(fut_sparse_ix, fut_ix_fit)]
fut_sparse_ix = fut_ix_fit
lab = np.array(lab)
### end Block 8

Here, the outcomes in each time period, again concatenated with the `cens_info` matrix to more easily see what each row of the fut_inc represents. For example, for subject 7296:
- they had no emphysema in the first interval (row 56, entry value is 0 for emphysema)
- then they have incident emphysema (row 57, entry is 1 for incident)
- then they have any time after that a history of emphysema, such as in row 58, represented by -1

In [26]:
pd.concat((cens_info, fut_inc), axis=1)[55:65]

,ids,censored,interval_start,interval_end,Arterial_Embolism_Thrombosis,E Codes: Other specified; NEC,Emphysema_COPD,General_Dementia,General_Paralysis,HIV,...,Pancytopenia,Parkinsons_Disease,Personality_Disorder,Septic_Arthritis,Septicemia,Spleen_Disease,Ulcerative_Colitis,Unspecified_Autoimmune_Disease,Unspecified_Hepatitis,Unspecified_Immune_Deficiency
55,4471.0,1,72.0,96.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,7296.0,0,0.0,24.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,7296.0,0,24.0,48.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
58,7296.0,1,48.0,72.0,0,0,-1,0,0,0,...,0,0,0,0,-1,0,0,0,0,0
59,12163.0,1,0.0,24.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,12527.0,0,0.0,24.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,12527.0,0,24.0,48.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,12527.0,0,48.0,72.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,12527.0,0,72.0,96.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,12527.0,0,96.0,120.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Block 9: fit censoring model and create weights
Now, we fit the weights for modeling probability of censoring.

This uses:
- `fut`: time specific medical record
- `past`: medical history at baseline (initiation of drug A)
- `dense`: demographics, etc
- `cens_info`: info about the follow up time
- `lab`: label (censored or not)

This adds columns to the `cens_info` so each person-time observation has the censoring weights and outcomes

In [28]:
### begin Block 9
discont_file = run_save + "bz2"

### make discsontinuation censoring: censor those who have not taken drug A recently
### to avoid confounding, make model of probabiklity of censoring to weight person-time
if not os.path.exists(discont_file):
    if (lab==1).sum() > 0:
        wt.p2weight(cens_info, lab, dense, past, fut,
                    ch_len/(TIME_CHUNK*agg),
                    run_save, past_sparse_index, fut_sparse_ix,
                    p='discont')
    else:
        cens_info['discont.cum_wt']= np.ones(cens_info.shape[0])
    #### save/load here
    ### cancer outcomes is stored in the data structure, just need to recover names..
    ids, outcome_inc = his2ft.load_outcomes(his2ft.get_h5(hisdir, drugid), np.loadtxt(fname), len(outcomes))
    outcome_inc = pd.DataFrame(outcome_inc,index=np.hstack(ids),
                               columns=['deenroll'] + outcomes + ['any_outcome'])
    ### now, we have a matrix with the rows = person-time uncensored, columns = censoring weights + outcomes 
    cens_info = wt.add_outcomes(outcome_inc, cens_info, TIME_CHUNK*agg)
    cens_info = cens_info.drop(['Cancer_history','Unspecified_Cancer','Obs_uspected_neoplasm','Secondary_Malignant_Neoplasm','any_outcome'],axis=1)
    cens_info.to_csv(discont_file,sep="\t",header=True,compression='bz2')
else:
    cens_info = pd.read_csv(discont_file,index_col=0,sep="\t")


In [29]:
cens_info.shape

(110990, 39)

So here the `cens_info` contains the previous interval info, plus the outcomes at each time interval (0,1,-1, as discussed above in block 7), and columns for the weights:
- `discont.den`: probability of censored, given current and baseline medical record
- `discont.num`: probability of censored, given baseline medical record
- `discont.single_wt`: $ {discont.num^{censored}\times (1-discont.num)^{1-censored}} \over
                        {discont.den^{censored}\times (1-discont.den)^{1-censored}} $
- `discont.cum_wt`: the product of all of the previous single_wt, representing total probability of censoring history

In [30]:
cens_info.head()

,ids,censored,interval_start,interval_end,discont.den,discont.num,discont.single_wt,discont.cum_wt,Bladder_Cancer,Bone_Cancer,...,Male_Genital_Cancer,Oro_Naso_Pharyngeal_Cancer,Other_Lymphoid_Histiocytic_Cancers,Ovarian_Cancer,Pancreatic_Cancer,Prostate_Cancer,Respiratory_System_Cancer,Stomach_Cancer,Thyroid_Cancer,Uterine_Cancer
0,6563.0,0,0.0,24.0,0.494681,0.544786,0.900844,0.900844,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6563.0,0,24.0,48.0,0.456909,0.472775,0.970786,0.874527,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6563.0,0,48.0,72.0,0.454682,0.399308,1.101543,0.963329,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6563.0,1,72.0,96.0,0.276585,0.293827,1.062339,1.023382,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14446.0,0,0.0,24.0,0.448866,0.592974,0.738524,0.738524,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Block 10: remove censored (discontinued) time points
We needed to keep in the censored points at time of censoring in order to fit the model. Now remove them.

In [35]:
### block 10
## remove censored (discontinued)time points
uncensored = np.where(lab==0)[0]
cens_info = cens_info.iloc[uncensored,:]
ch_len = ch_len[uncensored]
dense =  dense[uncensored,:];  past=past[uncensored,:]; fut = fut[uncensored,:]
fut_inc = fut_inc.iloc[uncensored,:] 
### end block 10


## 3. Drug combination analysis
Now we have our censoring set up and modeled for confounding. We can look at follow up time that is uncensored to see when drug 2 happens. 

We already found the drugs that have a sufficient number of users in Block 6. We do a "for loop" through these drugs. 

`for drug2 in todo:`

In [37]:
drug2 = 1190

#### Block 11: obtain people who either never take drug B or initiate after drug A
This code cleans up the data and pulls out people who are eligible for this "study".

Then, for the purpose of fitting the drug initiation model, we keep the subset of people who are eligible for initiating drug B = they have no history of drug B either before or after drug A, up through this time point.

In [40]:
drug2f = run_save + str(drug2) + "." 

combhis = cens_info.copy()            
todrop = set(['censored','discont.den','discont.num','discont.single_wt']) & set(combhis.columns)
if todrop:
    combhis = combhis.drop(todrop,axis=1).copy()

## add in if take drug2 in this period
combhis['drug2'] = np.array(fut[:,np.where(fut_sparse_ix==drug2)[0]].todense())[:,0]

init_weights = drug2f + "cens_trt.bz2" ## will have drug 2 init treat, init drug2-weights
### prevdrug = any time point after init d2. These points will be kept in as follow-up time but not weighted or fit in model
prevdrug = combhis.groupby('ids')['drug2'].transform(lambda  x: list(np.cumsum(np.array([0]+list(x.values[:-1])))))
pastdrug = np.array(past[:,np.where(past_sparse_index==drug2)[0][0]].todense())[:,0]

### only want to fit model on isntances with no previous drug2
### -- this means not before drug 1; and not after drug2 started
sel =  np.where((prevdrug==0) & (pastdrug==0))[0]
comb_init = []

comb_init = combhis.iloc[sel,:].copy()


This obtains:
- `combhis` = the subset with no history of drug B before drug A, so are eligible, including their follow-up time after inititin the ocmbination
- `comb_init`: for fitting the propensity score weighting, we keep only the person-time points up through trial start (not follow-up time after exposure to the combination, since this time the person is not eligible to start the drug). We use this matrix for the modeling.

In [42]:
combhis.head()

,ids,interval_start,interval_end,discont.cum_wt,Bladder_Cancer,Bone_Cancer,Brain_Cancer,Breast_Cancer,CNS_Cancer,Carinoid_Neoplasm,...,Oro_Naso_Pharyngeal_Cancer,Other_Lymphoid_Histiocytic_Cancers,Ovarian_Cancer,Pancreatic_Cancer,Prostate_Cancer,Respiratory_System_Cancer,Stomach_Cancer,Thyroid_Cancer,Uterine_Cancer,drug2
0,6563.0,0.0,24.0,0.900844,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6563.0,24.0,48.0,0.874527,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6563.0,48.0,72.0,0.963329,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14446.0,0.0,24.0,0.738524,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,18588.0,0.0,24.0,0.735229,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
comb_init.head()

,ids,interval_start,interval_end,discont.cum_wt,Bladder_Cancer,Bone_Cancer,Brain_Cancer,Breast_Cancer,CNS_Cancer,Carinoid_Neoplasm,...,Oro_Naso_Pharyngeal_Cancer,Other_Lymphoid_Histiocytic_Cancers,Ovarian_Cancer,Pancreatic_Cancer,Prostate_Cancer,Respiratory_System_Cancer,Stomach_Cancer,Thyroid_Cancer,Uterine_Cancer,drug2
0,6563.0,0.0,24.0,0.900844,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6563.0,24.0,48.0,0.874527,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6563.0,48.0,72.0,0.963329,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14446.0,0.0,24.0,0.738524,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,49817.0,0.0,24.0,0.916923,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
comb_init.shape

(46044, 36)

In [43]:
combhis.shape

(77463, 36)

#### Block 12: fit propensity weights for initiation of drug B
Remove drug B use as a feature, then just as with the censoring weights, we feed in the same sets of features for building a model to predict drug B initiation.


In [45]:
### Block 12
fut_columns =  fut_sparse_ix!=drug2
print("Weights for "+ drug2f  + " {:d}x{:d}".format(comb_init.shape[0],fut.shape[1] +past.shape[1]))
fut_do = fut[sel,:][:,np.where(fut_columns)[0]]
wt.p2weight(comb_init, comb_init['drug2'].values, dense[sel,:], past[sel,:],
            fut_do,
            ch_len[sel]/(TIME_CHUNK*agg),
             drug2f, past_sparse_index, fut_sparse_ix[fut_columns],p="drug2")
comb_init = comb_init.drop(['drug2.den','drug2.num'],axis=1)
### end Block 12


Weights for dcomb/Combo2.4565/0.agg-2.drug1-20.1190. 46044x20268
FILTERING ultrasparse MSM data: 13356 -> (46044, 3249)  for  dcomb/Combo2.4565/0.agg-2.drug1-20.1190.
get_weights: dcomb/Combo2.4565/0.agg-2.drug1-20.1190.drug2.den.pkl
FOLD  0  best= 0.3-0.001  @0.81 - Mem = 1.06Gb
FOLD  1  best= 0.3-0.001  @0.79 - Mem = 1.13Gb
FOLD  2  best= 0.3-0.001  @0.76 - Mem = 1.04Gb
FOLD  3  best= 0.3-0.001  @0.77 - Mem = 1.16Gb
FOLD  4  best= 0.3-0.001  @0.78 - Mem = 1.19Gb
get_weights: dcomb/Combo2.4565/0.agg-2.drug1-20.1190.drug2.num.pkl
FOLD  0  best= 0.3-0.001  @0.58 - Mem = 1.06Gb
FOLD  1  best= 0.3-0.001  @0.58 - Mem = 1.11Gb
FOLD  2  best= 0.3-0.001  @0.58 - Mem = 1.01Gb
FOLD  3  best= 0.3-0.001  @0.57 - Mem = 1.11Gb
FOLD  4  best= 0.3-0.001  @0.60 - Mem = 1.01Gb


In [46]:
comb_init.head()

,ids,interval_start,interval_end,discont.cum_wt,Bladder_Cancer,Bone_Cancer,Brain_Cancer,Breast_Cancer,CNS_Cancer,Carinoid_Neoplasm,...,Ovarian_Cancer,Pancreatic_Cancer,Prostate_Cancer,Respiratory_System_Cancer,Stomach_Cancer,Thyroid_Cancer,Uterine_Cancer,drug2,drug2.single_wt,drug2.cum_wt
0,6563.0,0.0,24.0,0.900844,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.728805,0.728805
1,6563.0,24.0,48.0,0.874527,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.788867,0.574930
2,6563.0,48.0,72.0,0.963329,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.764312,0.439426
4,14446.0,0.0,24.0,0.738524,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.553268,2.553268
9,49817.0,0.0,24.0,0.916923,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.693591,0.693591


#### Block 13: creating the final cumulative weights for the  model
Now that we have weighted the initiation times, we fill in the follow up time after initiating drug B with 1's (becuase they can't initiate after they have already initiated). Then we create the cumulative weights just as above by the product of the weights from the past history

In [47]:

### Block 13
### fixing the fact that there will be rows that were dropped for
### fitting the model.
init_weight = np.ones(combhis.shape[0])
init_weight[sel] = comb_init['drug2.single_wt']
print("Trials for "+ drug2f,combhis.shape)
combhis['init_weight'] = init_weight

## check someone who innitiated after time 1 but before  last time
combhis['drug2.cum_wt'] = combhis.groupby('ids')['init_weight'].agg('cumprod')
del comb_init
combhis = combhis.loc[pastdrug==0,:]
### end Block 13


Trials for dcomb/Combo2.4565/0.agg-2.drug1-20.1190. (77463, 36)


In [48]:
combhis.head()

,ids,interval_start,interval_end,discont.cum_wt,Bladder_Cancer,Bone_Cancer,Brain_Cancer,Breast_Cancer,CNS_Cancer,Carinoid_Neoplasm,...,Ovarian_Cancer,Pancreatic_Cancer,Prostate_Cancer,Respiratory_System_Cancer,Stomach_Cancer,Thyroid_Cancer,Uterine_Cancer,drug2,init_weight,drug2.cum_wt
0,6563.0,0.0,24.0,0.900844,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.728805,0.728805
1,6563.0,24.0,48.0,0.874527,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.788867,0.574930
2,6563.0,48.0,72.0,0.963329,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.764312,0.439426
4,14446.0,0.0,24.0,0.738524,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.553268,2.553268
9,49817.0,0.0,24.0,0.916923,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.693591,0.693591


#### Block 14: expanding into trials of person-time
We create an indexer that allows us to expand the matrix into repeated person-time.
For each person, we find if they have initiated yet. Once they initiate that is their last trial. Otherwise, every follow-up time is another trial.

In [49]:
### Block 14
### expanding into trials of person-time
trials = []
dsel = []
treated = []
## for each person, obtain their trials, up through the initiation of drug B each time is a trial
for pat in set(combhis['ids']):
    patrows = np.where(combhis['ids']==pat)[0]
    patdat = combhis.iloc[patrows,:]
    for trial in range(patdat.shape[0]):
        nr = patdat.shape[0] - trial
        trials += [trial]*nr
        trial_treated = patdat['drug2'].iloc[trial]
        treated += [trial_treated]*nr
        dsel += list(patrows[trial:patdat.shape[0]])
        ### no longer "eligible" if treated  here
        if trial_treated:
            break
print("Expanding "+ drug2f +" to ",len(dsel))
### end Block 14            


Expanding dcomb/Combo2.4565/0.agg-2.drug1-20.1190. to  164846


In [51]:
trials[:10]

[0, 0, 0, 0, 1, 1, 1, 2, 2, 3]

In [52]:
dsel[:10]

[14040, 14041, 14042, 14043, 14041, 14042, 14043, 14042, 14043, 14043]

#### Block 15: create final files
Using the indexers created above, we expand the follow up time to create the files for the survival analysis step.

In [53]:
### Block 15
fut_inc.loc[pastdrug==0,:].iloc[dsel,:].to_csv(drug2f + ctl_outcomes_name + ".bz2", sep="\t",header=True,compression="bz2", index=False)
trial_file = drug2f + "trials.bz2"
if not os.path.exists(trial_file):
    #pdb.set_trace()
    combhis = combhis.iloc[dsel,:]
    combhis['trial'] = trials
    combhis['treatment'] = treated
    if drug1_washout==np.inf:
        combhis['totwt'] = combhis['drug2.cum_wt']
    else:
        combhis['totwt'] = combhis['discont.cum_wt']*combhis['drug2.cum_wt']

    to_drop = set(['drug2','discont.cum_wt','drug2.cum_wt','init_weight']) & set(combhis.columns)
    combhis.drop(to_drop,axis=1).to_csv(trial_file,sep="\t",header=True,compression="bz2",index=False)
### end Block 15

Below you can see some examples of person-time:
- person 538 initiates the combination in trial 1
- person 542 has incident bladder cancer in the first trial (but is still in the data set, no history of other cancers at that time).

In [78]:
combhis.iloc[5650:5670,np.append(np.arange(4),np.arange(33,37))]

,ids,interval_start,interval_end,Bladder_Cancer,Uterine_Cancer,trial,treatment,totwt
5650,537.0,168.0,192.0,0,0,3,1,0.723673
5651,537.0,192.0,216.0,0,0,3,1,0.779567
5652,538.0,0.0,24.0,0,0,0,0,0.803905
5653,538.0,24.0,48.0,0,0,0,0,0.559707
5654,538.0,48.0,72.0,0,0,0,0,0.416699
5655,538.0,72.0,96.0,0,0,0,0,0.340390
5656,538.0,96.0,120.0,0,0,0,0,0.311450
5657,538.0,24.0,48.0,0,0,1,1,0.559707
5658,538.0,48.0,72.0,0,0,1,1,0.416699
5659,538.0,72.0,96.0,0,0,1,1,0.340390


In [82]:
fut_inc.loc[pastdrug==0,:].iloc[dsel,:].to_csv("example_data." + ctl_outcomes_name + ".bz2", sep="\t",header=True,compression="bz2", index=False)            

In [79]:
combhis.to_csv("example_data.trials.bz2",sep="\t",header=True,compression='bz2',index=False)

From here, the text  files are loaded into R:
```
source("survival_combo_effect.R")
runone("example_data.trials.bz2", "ctloutcomes_match", duUNWT=F)
```

The key parts of that code are:
- We load in the data and for each outcome, obtain only the time intervals where the person is at risk for the outcome (filtering out the "-1" rows such as row 5667 and 5668 if we were testing the effect on bladder cancer.
- We clip the weights, using a maximum weight of 5 for a person-time
- We run the surival analysis using hte interval format, where time to outcome is modeled as a function of treatment (0 for no combination, 1 for combination), and observations are weighted using our final weights `totwt`.  We use the `cluster` option ot account for the same patient being observed in multiple trials.
- We obtain the robust standard errors and effect estimate using the survival and survey packages.